# Custom Chatbot Notebook

An OpenAI client is initialised by using environment variables and a tokenizer is set up for a specific model (`gpt-4o-mini-2024-07-18`). Also,the necessary libraries and custom utility functions are imported.

In [28]:
import pandas as pd
import os
from pathlib import Path
from dotenv import load_dotenv
import tiktoken

# Custom Functions
from fncs.utilities import (
    create_openai_client,
    response_generator,
    prompt_builder,
    calculate_total_cost
    )
from fncs.retrieval import (
    get_embedding,
    search_text,
    control_chunk_context
    )

# Load environment vars:
load_dotenv()
base_url_voc = os.getenv("OPENAI_BASE_VOC")
api_key_voc = os.getenv("OPENAI_API_VOC")
# Deployment model names
chat_name = 'gpt-4o-mini' # 'gpt-4o-mini-2024-07-18' # 'gpt-4o-mini'
emb_name = 'text-embedding-3-large'
# Initialising OpenAI client
openai_client = create_openai_client(api_key= api_key_voc, base_url= base_url_voc)
tokenizer = tiktoken.encoding_for_model(chat_name)

### Loading dataset

In [29]:
proj_dir = Path(os.getcwd())
df = pd.read_csv(proj_dir / "data" / "2023_fashion_trends_embeddings.csv")
df.head(3)

,text,embeddings
0,Title: 7 Fashion Trends That Will Take Over 20...,"[-0.06084602698683739, -0.00787690281867981, -..."
1,Title: 7 Fashion Trends That Will Take Over 20...,"[-0.06700262427330017, -0.014003804884850979, ..."
2,Title: 7 Fashion Trends That Will Take Over 20...,"[-0.05102064833045006, -0.00858586560934782, -..."


The embeddings are stored as text/string in the DataFrame and need to be converted to lists/arrays

In [30]:
import ast
# Converting the string representations of embeddings to actual lists
df['embeddings'] = df['embeddings'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

Checking transformation

In [31]:
type(df[['embeddings']].iloc[0].values[0])

list

### Calculating Cosine Distances based on query


Below I create a query string about fashion trends in 2023. Then, by using the `get_embedding` function, the embeddings of the query are generated, by passing the query, OpenAI client, and embedding model as inputs.

In [32]:
query = "What is the most popular fashion trend about pants in 2023?"
query_emb = get_embedding(text=query, client = openai_client, model=emb_name)

The DataFrame `df` is sorted based on the cosine distance between the query embedding (`query_emb`) and the embeddings in the DataFrame using the `search_text` function, and stores the result in `df_sorted`.

In [33]:
df_sorted = search_text(df=df, embs_query=query_emb, cosine='distance')

In [34]:
df_sorted.head()

,text,embeddings,distance
1,Title: 7 Fashion Trends That Will Take Over 20...,"[-0.06700262427330017, -0.014003804884850979, ...",0.273721
3,Title: 7 Fashion Trends That Will Take Over 20...,"[-0.05067730322480202, -0.02512504905462265, -...",0.307084
58,Title: Spring/Summer 2023 Fashion Trends: 21 E...,"[-0.03485928103327751, -0.015784457325935364, ...",0.309776
44,Title: Spring/Summer 2023 Fashion Trends: 21 E...,"[-0.04672637954354286, -0.03269721940159798, -...",0.310095
19,Title: 9 Spring 2023 Fashion Trends You’ll Wan...,"[-0.04425228014588356, -0.035396404564380646, ...",0.355568


### Prompt Template



Creating the system prompt to be used in the chatbot

In [35]:
system_prompt = "You are an expert fashion trend analyser. Based only on the provided information you must analyse and summarise the trends and provide an accurate answer."

print(f"System Prompt Tokens: {len(tokenizer.encode(system_prompt))}")

System Prompt Tokens: 28


Creating the user prompt to be used in the chatbot

In [36]:
user_prompt = \
"""
***Question: {}

***Context:
<--Start of Context-->
{}
<--End of Context-->

**Instructions:
- Answer based ONLY on the provided context above
- Do not include external knowledge
- Be concise and specific

**Required Format:
1. Answer:
   [Your detailed response here]

2. Key Points:
   • [Bullet point 1]
   • [Bullet point 2]
   • [...]

3. Sources:
   • [Source URL 1]
   • [Source URL 2]

Note: If the answer cannot be determined from the provided context,
state: "Cannot be determined from the given context."
"""
print(f"User Prompt Tokens BEFORE context insertion: {len(tokenizer.encode(user_prompt))}")

User Prompt Tokens BEFORE context insertion: 130


In [37]:
# to be used in performance demonstration later
user_prompt_without_context = \
"""
***Question: {}

**Instructions:
- Be concise and specific

**Required Format:
1. Answer:
   [Your detailed response here]

2. Key Points:
   • [Bullet point 1]
   • [Bullet point 2]
   • [...]

3. Sources:
   • [Source URL 1]
   • [Source URL 2]
"""
print(f"User Prompt Tokens BEFORE context insertion: {len(tokenizer.encode(user_prompt))}")

User Prompt Tokens BEFORE context insertion: 130


#### Apply token controller function ( fnc: control_chunk_context )

The variable `max_token_count` to 1000, serves as a limit for the total number of tokens allowed in a prompt.

In [38]:
#parameter that control the prompt tokens:
max_token_count = 1000

The code below calculates the current token count of the prompts (system and user) and generates a context by selecting data from the sorted DataFrame (`df_sorted`) based on a maximum allowed token limit (`max_token_count`) using the `control_chunk_context` function.

In [39]:
current_token_count = len(tokenizer.encode(user_prompt)) + len(tokenizer.encode(system_prompt))
# Create context from sorted dataframe according to the max token limit
context = control_chunk_context(
    df_sorted,
    current_token_count,
    max_token_count,
    tokenizer = tokenizer
)

 Below, the final `user_prompt` is created by inserting the generated `context` into the prompt template and by formatting it with the query and context.

In [40]:
# prompt template params
context_inprompt = "\n----\n".join(context)
user_prompt_0 = user_prompt.format(query, context_inprompt)

print(user_prompt)


***Question: {}

***Context:
<--Start of Context-->
{}
<--End of Context-->

**Instructions:
- Answer based ONLY on the provided context above
- Do not include external knowledge
- Be concise and specific

**Required Format:
1. Answer:
   [Your detailed response here]

2. Key Points:
   • [Bullet point 1]
   • [Bullet point 2]
   • [...]

3. Sources:
   • [Source URL 1]
   • [Source URL 2]

Note: If the answer cannot be determined from the provided context,
state: "Cannot be determined from the given context."



In [41]:
print(f"User Prompt Tokens AFTER context insertion: {len(tokenizer.encode(user_prompt))}")

User Prompt Tokens AFTER context insertion: 130


## Custom Query Completion

**Finally, the code below generates a final prompt using the `prompt_builder` function by combining the system and user prompts. It then sends the prompt to the OpenAI model (`chat_model`) using the `response_generator` function with specified additional options (e.g., `temperature=0`) to generate an AI response. It also calculates the total cost in EUR based on the API usage (`response_full.usage`) for the specific deployment (`gpt-4o-mini`).**

In [42]:
final_prompt = prompt_builder(system_content= system_prompt, user_content_prompt= user_prompt_0)
additional_options = {"temperature": 0.4,}
response, response_full = response_generator(openai_client, chat_model=chat_name, prompts=final_prompt, options=additional_options)
cost_eur = calculate_total_cost(response_usage= response_full.usage,
                                deployment_name= chat_name)
print(f'Query Completion Total Cost is: {cost_eur} eur')

Query Completion Total Cost is: 0.0002631777 eur


In [43]:
print(response)

1. Answer:
   The most popular fashion trend about pants in 2023 is the resurgence of cargo pants, which are being reimagined with tailored silhouettes, unique pocket placements, and luxurious fabrics. Additionally, baggy denim styles are also trending, with a focus on looser fits and timeless cuts. Overall, trousers are a significant focus for the season, with a variety of styles including wide-leg, puddle hemlines, and slouchy fits gaining popularity.

2. Key Points:
   • Cargo pants are making a comeback with tailored designs and elevated materials.
   • Baggy denim remains popular, featuring looser fits and versatile styling options.
   • The trend encompasses a variety of trouser styles, including wide-leg and puddle hemlines.

3. Sources:
   • www.refinery29.com
   • www.whowhatwear.com
   • www.glamour.com


In [44]:
print('Total Tokens: ', response_full.usage.total_tokens)
print('Total Completion Tokens: ', response_full.usage.completion_tokens)
print('Total Prompt Tokens: ', response_full.usage.prompt_tokens)

Total Tokens:  1110
Total Completion Tokens:  184
Total Prompt Tokens:  926


## Demonstrating Performance

Below, two questions (queries) are ... ...

### Question 1
**Question**: According to Vogue, what is a new trend presented by Prada on New York Fashion Week?

In [45]:
query_1 = "According to Vogue, what is a new trend presented by Prada on New York Fashion Week?"
max_token_count = 1000

In [46]:
query_emb = get_embedding(text=query_1, client = openai_client, model=emb_name)
df_sorted = search_text(df=df, embs_query=query_emb, cosine='distance')

current_token_count = len(tokenizer.encode(user_prompt)) + len(tokenizer.encode(system_prompt))
# Create context from sorted dataframe according to the max token limit
context = control_chunk_context(chunks_sorted_df=df_sorted,
                                current_token_count=current_token_count,
                                max_token_count=max_token_count,
                                tokenizer = tokenizer)
context_inprompt = "\n----\n".join(context)
user_prompt_1 = user_prompt.format(query_1, context_inprompt)

final_prompt = prompt_builder(system_content= system_prompt, user_content_prompt= user_prompt_1)
additional_options = {"temperature": 0,}

response_1_1, response_full_1_1 = \
    response_generator(openai_client, chat_model=chat_name, prompts=final_prompt, options= additional_options)

cost_eur_1_1 = \
    calculate_total_cost(response_usage= response_full.usage, deployment_name= chat_name)
print(f'Query Completion Total Cost is: {cost_eur_1_1} eur')


Query Completion Total Cost is: 0.0002631777 eur


In [47]:
print(response_1_1)

1. Answer:
   A new trend presented by Prada at New York Fashion Week is the "Perfectly Imperfect" style, characterized by a satin midi skirt that features an irregularly dyed print and a slit designed to give the appearance of being torn. This trend evokes a sense of "unfinishedness" in fashion.

2. Key Points:
   • Prada's satin midi skirt showcases an "unfinished" aesthetic.
   • The design includes an irregularly dyed print and a slit that mimics a torn look.

3. Sources:
   • www.vogue.com


In [48]:
final_prompt = prompt_builder(system_content= system_prompt, user_content_prompt= query_1) #or use: user_prompt_without_context.format(query_1)
additional_options = {"temperature": 0,}

response_1_2, response_full_1_2 = response_generator(openai_client, chat_model=chat_name, prompts=final_prompt, options=additional_options)
cost_eur_1_2 = calculate_total_cost(response_usage= response_full.usage,
                                deployment_name= chat_name)
print(f'Query Completion Total Cost is: {cost_eur_1_2} eur')

Query Completion Total Cost is: 0.0002631777 eur


In [49]:
print(response_1_2)

As of October 2023, Prada showcased a notable trend at New York Fashion Week that emphasizes a blend of sophistication and practicality. The collection featured a mix of tailored silhouettes with unexpected elements, such as bold colors and unique textures. This trend reflects a growing preference for versatile pieces that can transition from day to night, highlighting the importance of functionality in high fashion. Additionally, Prada's use of innovative materials and sustainable practices aligns with the broader industry movement towards eco-conscious fashion. Overall, the trend signifies a shift towards modern elegance that prioritizes both style and wearability.


### Question 2
**Question**:

In [50]:
query_2 = "What an indie sleaze is and how it affected the fashion trends of 2023?"
max_token_count = 1000

In [51]:
query_emb = get_embedding(text=query_2, client = openai_client, model=emb_name)
df_sorted = search_text(df=df, embs_query=query_emb, cosine='distance')

current_token_count = len(tokenizer.encode(user_prompt)) + len(tokenizer.encode(system_prompt))
# Create context from sorted dataframe according to the max token limit
context = control_chunk_context(chunks_sorted_df=df_sorted,
                                current_token_count=current_token_count,
                                max_token_count=max_token_count,
                                tokenizer = tokenizer)
context_inprompt = "\n----\n".join(context)
user_prompt_2 = user_prompt.format(query_2, context_inprompt)

final_prompt = prompt_builder(system_content= system_prompt, user_content_prompt= user_prompt_2)
additional_options = {"temperature": 0,}

response_2_1, response_full_2_1 = \
    response_generator(openai_client, chat_model=chat_name, prompts=final_prompt, options= additional_options)

cost_eur_2_1 = \
    calculate_total_cost(response_usage= response_full.usage, deployment_name= chat_name)
print(f'Query Completion Total Cost is: {cost_eur_2_1} eur')

Query Completion Total Cost is: 0.0002631777 eur


In [52]:
print(response_2_1)

1. Answer:
   Indie sleaze is a fashion aesthetic that draws inspiration from the edgy, carefree styles of the late 2000s and early 2010s, characterized by elements such as distressed denim, layered tops, and utilitarian details. In 2023, this trend has significantly influenced fashion, as seen in the spring/summer collections that feature muddy hues, oversized pockets, and cargo shapes. The resurgence of this nostalgic style aligns with other trends like sheer clothing, daytime shine, and reimagined denim, indicating a broader embrace of retro influences across various fashion categories.

2. Key Points:
   • Indie sleaze reflects a nostalgic return to edgy styles from the late 2000s and early 2010s.
   • Key elements include distressed denim, layered tops, and modern utility detailing.
   • The trend is part of a larger movement in 2023 that includes sheer clothing, daytime shine, and innovative denim styles.

3. Sources:
   • www.whowhatwear.com
   • www.refinery29.com


In [53]:
final_prompt = prompt_builder(system_content= system_prompt, user_content_prompt= query_2) #or use: user_prompt_without_context.format(query_2)
additional_options = {"temperature": 0,}

response_2_2, response_full_2_2 = response_generator(openai_client, chat_model=chat_name, prompts=final_prompt, options=additional_options)

cost_eur_2_2 = calculate_total_cost(response_usage= response_full.usage,deployment_name= chat_name)
print(f'Query Completion Total Cost is: {cost_eur_2_2} eur')

Query Completion Total Cost is: 0.0002631777 eur


In [54]:
print(response_2_2)

Indie sleaze is a fashion and cultural aesthetic that emerged in the early 2000s, characterized by a mix of grunge, punk, and vintage influences. It often features elements such as oversized clothing, thrifted pieces, graphic tees, skinny jeans, and a general DIY ethos. The look is often accessorized with items like beanies, chunky jewelry, and retro sunglasses, reflecting a carefree, rebellious attitude.

In 2023, indie sleaze made a notable comeback, influencing fashion trends significantly. This resurgence can be attributed to a nostalgia for early 2000s culture, driven by social media platforms like TikTok and Instagram, where vintage and retro styles are celebrated. Key trends that emerged from this revival include:

1. **Thrift Culture**: A renewed interest in second-hand shopping and sustainable fashion, with many consumers seeking unique, vintage pieces that embody the indie sleaze aesthetic.

2. **Layering and Oversized Silhouettes**: The trend embraced oversized jackets, bagg